In [11]:
import xml.etree.ElementTree as ET
import re
import os.path
from io import BytesIO

#tree = ET.parse(r'C:\Users\Chris\Documents\projects\train-game\drawing.svg')
infile = 'drawing.svg'
svg_outfile = os.path.join('.', 'public', 'ticket-nobg.svg')
react_outfile = os.path.join('.', 'public', 'ticket-nobg.react.svg')

ns = {'svg': 'http://www.w3.org/2000/svg',
      'ttr': 'https://train-game.github.io',
          'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

ET.register_namespace('', 'http://www.w3.org/2000/svg')
ET.register_namespace('inkscape', 'http://www.inkscape.org/namespaces/inkscape')
ET.register_namespace('sodipodi', 'http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd')

def fix_doc(node):
    # Remove namespaced attributes from node
    for attrib_name in list(node.attrib.keys()):
        if 'inkscape' in attrib_name or 'sodipodi' in attrib_name or 'XML' in attrib_name:
            del node.attrib[attrib_name]
    for child in node.findall('*'):
        # Strip nodes
        if 'inkscape' in child.tag or 'sodipodi' in child.tag or child.get('id') == 'BG':
            node.remove(child)
            continue

        # Rename inkscape:label to id
        label = child.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label')
        if label:
            child.set('id', label)
            del child.attrib['{http://www.inkscape.org/namespaces/inkscape}label']

        # Remove old styles
        try:
            del child.attrib['style']
        except KeyError:
            pass

    for child in node.findall('*'):
        fix_doc(child)

def add_additional_data_to_routes(node):
    """
    Things this method does to each route:
    
    1. Encode information into the ID
    2. add onClick attribute to the routes
    """
    ROUTE_ID_FORMAT_STRING = '{to_and_from}:{lane_index}:{color}:{length}'
    for route in node.find("*/[@id='Routes']").findall('*'): # All first-level routes (single routes + double routes)
        num_trains = len(route.findall('{http://www.w3.org/2000/svg}rect'))
        color = route.attrib.get('class', None)
        to_and_from = route.attrib['id']
    
        if num_trains > 0:
            if not color:
                color = "grey"
            new_id = ROUTE_ID_FORMAT_STRING.format(to_and_from=to_and_from, lane_index=0, color=color, length=num_trains)
            print(new_id)
            route.set('id', new_id)
            route.attrib['onClick'] = 'CLAIM_ROUTE'
        else:
            for ix, child in enumerate(route.findall('{http://www.w3.org/2000/svg}g')):
                num_trains = len(child.findall('{http://www.w3.org/2000/svg}rect'))
                if not color:
                    color = child.attrib.get('class', 'grey')  # cascade
                new_id = ROUTE_ID_FORMAT_STRING.format(to_and_from=to_and_from, lane_index=ix, color=color, length=num_trains)
                print(new_id)
                child.set('id', new_id)
                route.attrib['onClick'] = 'CLAIM_ROUTE'

def make_child_route_ids(node):
    for route in node.find("*/[@id='Routes']").findall('*'):
        print(route)
        for ix, child in enumerate(route.findall('{http://www.w3.org/2000/svg}g')):
            child.set('id', f"{route.attrib['id']}:{ix}")
            
def make_reacty(node):
    # Make styles react-y (part 1)
    style = node.attrib.get('style')
    if style:
        rules = style.split(';')
        new_rules = []
        for rule in rules:
            key, value = rule.split(':')
            new_key = re.sub(r'(-\w)', lambda result: result.group(1).replace('-', '').upper(), key)
            new_rules.append(f'{new_key}:"{value}"')
        
        node.attrib['style'] = '{{' + ','.join(new_rules) + '}}'
    
    for child in node.findall('*'):
        make_reacty(child)

def to_react_string(svg_node):
    svg_data = ET.tostring(root).decode()
    svg_data = re.sub(r'style="(.*?)"', lambda result: f"style={result.group(1)}", svg_data)
    svg_data = svg_data.replace(r'&quot;', '"')
    svg_data = svg_data.replace(r'&gt;', '>')
    svg_data = svg_data.replace('"CLAIM_ROUTE"', '{claimRoute}')
    return svg_data  # svg_data[:1000]


def add_on_click_to_routes(root):
    for route in root.find("*/[@id='Routes']").findall('*'):
        route.attrib['onClick'] = 'CLAIM_ROUTE'



def generate_route_json(root):
    route_list = []
    for route in root.find("*/[@id='Routes']").findall('*'):
        id = route.attrib['id']
        city1 = id.split('-')[0]
        city2 = id.split('-')[1]
        route_list.append({
            "id": id,
            "city1": city1,
            "city2": city2,
        })
    return json.dumps(route_list)


In [10]:
tree = ET.parse(infile)

root = tree.getroot()

fix_doc(root)
make_child_route_ids(root)
add_additional_data_to_routes(root)

# write fixed file
open(svg_outfile,'w').write(ET.tostring(root).decode())

# write reacty file
make_reacty(root)
open(react_outfile, 'w').write(to_react_string(root))

# write json file
# open(json_outfile, 'w').write(generate_route_json(root))

<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B82F70>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B82D90>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B82930>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B82700>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B80F90>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81DF0>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81E40>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81C10>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81990>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81170>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81440>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26B81580>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26A850D0>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26A85300>
<Element '{http://www.w3.org/2000/svg}g' at 0x0000024F26ABFE70>
<Element '{http://www.w3.org/2000/svg}g'

47730